In [1]:
import torch
import time
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from efficientnet_pytorch import EfficientNet
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [2]:
transform_train = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
    transforms.RandomResizedCrop(136, scale=(0.8, 1.0)),  # Randomly crop and resize
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),  # Randomly translate horizontally and vertically
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_val = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
# transform_train = transforms.Compose([
#     transforms.Resize((96, 96)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(15),
#     transforms.RandomVerticalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# transform_val = transforms.Compose([
#     transforms.Resize((96, 96)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

In [4]:
training_dataset = datasets.ImageFolder('train', transform=transform_train)
validation_dataset = datasets.ImageFolder('val', transform=transform_val)

training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [5]:
num_classes = 6  # Number of classes in your dataset

In [6]:
# model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_classes)

In [7]:
# Define the model
model = EfficientNet.from_pretrained('efficientnet-b6', advprop=True,  num_classes=num_classes)

Loaded pretrained weights for efficientnet-b6


In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        56, 56, kernel_size=(3, 3), stride=[1, 1], groups=56, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        56, 14, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        14, 56, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        56, 32, kernel_size=(1, 1), stride=(1, 1), bias=False
  

In [10]:
# Define lists to store metrics
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

# Set number of epochs
num_epochs = 30

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_loss = 0.0
    start_epoch_time = time.time()
    
    for images, labels in training_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()
        train_loss += loss.item()
    
    # Compute average training loss and accuracy
    train_accuracy = train_correct / len(training_loader.dataset)
    train_loss /= len(training_loader)
    epoch_time = time.time() - start_epoch_time
    
    # Append training loss and accuracy to lists
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    
    # Evaluate on validation set
    model.eval()
    val_correct = 0
    val_loss = 0.0
    total_samples = 0
    
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_loss += loss.item()
            total_samples += labels.size(0)
        
    # Compute average validation loss and accuracy
    val_accuracy = val_correct / total_samples
    val_loss /= len(validation_loader)
    
    # Append validation loss and accuracy to lists
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
          f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, '
          f'Time: {epoch_time:.2f} seconds')

total_training_time = time.time() - start_time
total_training_time_hours = total_training_time / 3600
print(f'Total Training Time: {total_training_time_hours:.2f} hours')

Epoch [1/30], Train Loss: 1.2109, Train Accuracy: 0.5707, Validation Loss: 1.2626, Validation Accuracy: 0.4888, Time: 348.33 seconds
Epoch [2/30], Train Loss: 0.9120, Train Accuracy: 0.6974, Validation Loss: 1.4365, Validation Accuracy: 0.5046, Time: 404.39 seconds
Epoch [3/30], Train Loss: 0.6964, Train Accuracy: 0.7630, Validation Loss: 1.3702, Validation Accuracy: 0.6192, Time: 698.99 seconds


KeyboardInterrupt: 

In [ ]:
# Plot training loss
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color='blue')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot training accuracy
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy', color='green')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics for training data
train_preds = []
train_targets = []

model.eval()
with torch.no_grad():
    for images, labels in training_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        train_preds.extend(predicted.cpu().numpy())
        train_targets.extend(labels.cpu().numpy())

train_acc = accuracy_score(train_targets, train_preds)
train_precision = precision_score(train_targets, train_preds, average='weighted')
train_recall = recall_score(train_targets, train_preds, average='weighted')
train_f1 = f1_score(train_targets, train_preds, average='weighted')

print("Training Metrics:")
print(f"Accuracy: {train_acc:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1-score: {train_f1:.4f}")

In [ ]:
# Calculate metrics for validation data
val_preds = []
val_targets = []

with torch.no_grad():
    for images, labels in validation_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        val_preds.extend(predicted.cpu().numpy())
        val_targets.extend(labels.cpu().numpy())

val_acc = accuracy_score(val_targets, val_preds)
val_precision = precision_score(val_targets, val_preds, average='weighted')
val_recall = recall_score(val_targets, val_preds, average='weighted')
val_f1 = f1_score(val_targets, val_preds, average='weighted')

print("\nValidation Metrics:")
print(f"Accuracy: {val_acc:.4f}")
print(f"Precision: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")
print(f"F1-score: {val_f1:.4f}")

In [ ]:
conf_matrix_val = confusion_matrix(val_targets, val_preds)

# Print and plot confusion matrix for validation data
print("Confusion Matrix (Validation Data):")
print(conf_matrix_val)

plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix_val, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Validation Data)')
plt.colorbar()
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()